In [21]:
import warnings
from datetime import datetime, timedelta
from pathlib import Path

import cartopy.crs as ccrs
import cartopy.util
import dateutil.parser
import iris
import iris.coord_categorisation
import iris.pandas
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib.dates import DateFormatter
from matplotlib.ticker import FuncFormatter
from shapely.geometry import Point, Polygon

In [22]:
from util_commons import EXPERIMENTS, GASES
from util_mypaths import path_to_atom, path_to_processed, path_to_raw

In [23]:
warnings.filterwarnings("ignore", module="iris")
warnings.filterwarnings("ignore", module="xarray")
plt.rcParams["mathtext.default"] = "regular"

In [24]:
# Switches
atom_yrmn = 1608  # 1608 or 1702
ukca_mn = "Aug"  # Aug or Feb
t0, t1 = 12, 120  # used in thesis: 24, 120

#### Read data

In [25]:
# Select flights
fname_atom = f"MER-WAS_DC8_20{atom_yrmn}*.nc"
# Read ATom data
kw_mf = dict(combine="nested", concat_dim="time", decode_cf=True)
atom_dsinf = xr.open_mfdataset(sorted(path_to_atom.glob(fname_atom)), decode_times=True)
atom_dsmms = xr.open_mfdataset(sorted(path_to_atom.glob(fname_atom)), group="MMS", **kw_mf)
atom_dswas = xr.open_mfdataset(sorted(path_to_atom.glob(fname_atom)), group="WAS", **kw_mf)
atom_dsch4 = xr.open_mfdataset(sorted(path_to_atom.glob(fname_atom)), group="NOAA-Picarro", **kw_mf)
atom_dsno = xr.open_mfdataset(sorted(path_to_atom.glob(fname_atom)), group="NOyO3-NO", **kw_mf)
atom_dsno2 = xr.open_mfdataset(sorted(path_to_atom.glob(fname_atom)), group="NOyO3-NO2", **kw_mf)
atom_dsnoy = xr.open_mfdataset(sorted(path_to_atom.glob(fname_atom)), group="NOyO3-NOy", **kw_mf)
atom_dso3 = xr.open_mfdataset(sorted(path_to_atom.glob(fname_atom)), group="NOyO3-O3", **kw_mf)
atom_dsoh = xr.open_mfdataset(sorted(path_to_atom.glob(fname_atom)), group="ATHOS-HOx", **kw_mf)

In [6]:
# Choose experiment
exp = "CHEM"  # BASE; DONE CHEM, MARI, FIRE, FULL
path_to_exp = path_to_processed / EXPERIMENTS[exp] / "releveled"
fname_ukca = f"{EXPERIMENTS[exp]}_relvl"

In [7]:
# Read UKCA data
cb_nyrs_ch4 = iris.load_cube(str(path_to_exp / (fname_ukca + "_ch4.nc")), "ch4")
cb_nyrs_c2h6 = iris.load_cube(str(path_to_exp / (fname_ukca + "_c2h6.nc")), "c2h6")
cb_nyrs_c3h8 = iris.load_cube(str(path_to_exp / (fname_ukca + "_c3h8.nc")), "c3h8")

cb_nyrs_no = iris.load_cube(str(path_to_exp / (fname_ukca + "_no.nc")), "no")
cb_nyrs_no2 = iris.load_cube(str(path_to_exp / (fname_ukca + "_no2.nc")), "no2")
cb_nyrs_hono = iris.load_cube(str(path_to_exp / (fname_ukca + "_hono.nc")), "hono")
cb_nyrs_ho2no2 = iris.load_cube(str(path_to_exp / (fname_ukca + "_ho2no2.nc")), "ho2no2")
cb_nyrs_hno3 = iris.load_cube(str(path_to_exp / (fname_ukca + "_hno3.nc")), "hno3")
cb_nyrs_n2o5 = iris.load_cube(str(path_to_exp / (fname_ukca + "_n2o5.nc")), "n2o5")
cb_nyrs_pan = iris.load_cube(str(path_to_exp / (fname_ukca + "_pan.nc")), "pan")
cb_nyrs_ppan = iris.load_cube(str(path_to_exp / (fname_ukca + "_ppan.nc")), "ppan")

cb_nyrs_meono2 = iris.load_cube(str(path_to_exp / (fname_ukca + "_meono2.nc")), "meono2")
cb_nyrs_etono2 = iris.load_cube(str(path_to_exp / (fname_ukca + "_etono2.nc")), "etono2")
cb_nyrs_nprono2 = iris.load_cube(str(path_to_exp / (fname_ukca + "_nprono2.nc")), "nprono2")
cb_nyrs_iprono2 = iris.load_cube(str(path_to_exp / (fname_ukca + "_iprono2.nc")), "iprono2")

cb_nyrs_o3 = iris.load_cube(str(path_to_exp / (fname_ukca + "_o3.nc")), "o3")
cb_nyrs_oh = iris.load_cube(str(path_to_exp / (fname_ukca + "_oh.nc")), "oh")

# Read UKCA horizontal coordinates
ukca_lats = iris.load_cube(
    str(path_to_raw / "um_orography_xnvtj.nc"), "OROGRAPHY (/STRAT LOWER BC)"
).coord("latitude")
ukca_lons = iris.load_cube(
    str(path_to_raw / "um_orography_xnvtj.nc"), "OROGRAPHY (/STRAT LOWER BC)"
).coord("longitude")

#### Select geographical regions

In [8]:
# Extract datetimes from ATom data
atom_datetime = atom_dsinf.time.values.astype("<M8[us]").astype(datetime)
atom_date_strt = atom_datetime[0]
atom_date_stop = atom_datetime[-1]
# Extract spatial coordinates from ATom data
sample_lats = np.asarray(atom_dsmms.G_LAT.data)
sample_lons = np.asarray(atom_dsmms.G_LONG.data)
sample_alts = np.asarray(atom_dsmms.G_ALT.data)
# Construct pairs of coordinate points
sample_lon_lat_points = []
for i, j in zip(sample_lons, sample_lats):
    sample_lon_lat_points.append(Point(i, j))

In [9]:
# Define a geographical region
r1_llon_ukca, r1_ulon_ukca = 195.9375, 214.6875
r1_llon, r1_ulon = r1_llon_ukca - 360, r1_ulon_ukca - 360
r1_llat, r1_ulat = 21.875, 53.125
r1 = Polygon([(r1_llon, r1_llat), (r1_llon, r1_ulat), (r1_ulon, r1_ulat), (r1_ulon, r1_llat)])
r2_llon_ukca, r2_ulon_ukca = 169.6875, 201.5625
r2_1_llon, r2_1_ulon = r2_llon_ukca, 180
r2_2_llon, r2_2_ulon = -180, r2_ulon_ukca - 360
r2_llat, r2_ulat = -40.625, 20.625
r2_1 = Polygon(
    [(r2_1_llon, r2_llat), (r2_1_llon, r2_ulat), (r2_1_ulon, r2_ulat), (r2_1_ulon, r2_llat)]
)
r2_2 = Polygon(
    [(r2_2_llon, r2_llat), (r2_2_llon, r2_ulat), (r2_2_ulon, r2_ulat), (r2_2_ulon, r2_llat)]
)
r3_llon_ukca, r3_ulon_ukca = 180.9375, 285.9375
r3_llon, r3_ulon = r3_llon_ukca - 360, r3_ulon_ukca - 360
r3_llat, r3_ulat = -70.625, -50.625
r3 = Polygon([(r3_llon, r3_llat), (r3_llon, r3_ulat), (r3_ulon, r3_ulat), (r3_ulon, r3_llat)])
r4_llon_ukca, r4_ulon_ukca = 312.1875, 334.6875
r4_llon, r4_ulon = r4_llon_ukca - 360, r4_ulon_ukca - 360
r4_llat, r4_ulat = -45.625, -25.625
r4 = Polygon([(r4_llon, r4_llat), (r4_llon, r4_ulat), (r4_ulon, r4_ulat), (r4_ulon, r4_llat)])
r5_llon_ukca, r5_ulon_ukca = 323.4375, 344.0625
r5_llon, r5_ulon = r5_llon_ukca - 360, r5_ulon_ukca - 360
r5_llat, r5_ulat = -6.875, 36.875
r5 = Polygon([(r5_llon, r5_llat), (r5_llon, r5_ulat), (r5_ulon, r5_ulat), (r5_ulon, r5_llat)])
r6_llon_ukca, r6_ulon_ukca = 321.5625, 338.4375
r6_llon, r6_ulon = r6_llon_ukca - 360, r6_ulon_ukca - 360
r6_llat, r6_ulat = 38.125, 63.125
r6 = Polygon([(r6_llon, r6_llat), (r6_llon, r6_ulat), (r6_ulon, r6_ulat), (r6_ulon, r6_llat)])
r7_llon_ukca, r7_ulon_ukca = 269.0625, 321.5625
r7_llon, r7_ulon = r7_llon_ukca - 360, r7_ulon_ukca - 360
r7_llat, r7_ulat = 59.375, 81.875
r7 = Polygon([(r7_llon, r7_llat), (r7_llon, r7_ulat), (r7_ulon, r7_ulat), (r7_ulon, r7_llat)])
# r8_llon_ukca, r8_ulon_ukca = 237.1875, 269.0625
# r8_llon, r8_ulon = r8_llon_ukca-360, r8_ulon_ukca-360
# r8_llat, r8_ulat = 31.875, 50.625
# r8 = Polygon([(r8_llon, r8_llat), (r8_llon, r8_ulat), (r8_ulon, r8_ulat), (r8_ulon, r8_llat)])
r9_llon_ukca, r9_ulon_ukca = (
    201.5625,
    233.4375,
)  # between 234 and 237.1875 there is a spike in alkanes!
r9_llon, r9_ulon = r9_llon_ukca - 360, r9_ulon_ukca - 360
r9_llat, r9_ulat = 61.875, 81.875
r9 = Polygon([(r9_llon, r9_llat), (r9_llon, r9_ulat), (r9_ulon, r9_ulat), (r9_ulon, r9_llat)])
# Find points within a region
r1_points_within = []
r2_1_points_within, r2_2_points_within = [], []
r3_points_within = []
r4_points_within = []
r5_points_within = []
r6_points_within = []
r7_points_within = []
# r8_points_within = []
r9_points_within = []
for p in sample_lon_lat_points:
    r1_points_within.append(p.within(r1))
    r2_1_points_within.append(p.within(r2_1))
    r2_2_points_within.append(p.within(r2_2))
    r3_points_within.append(p.within(r3))
    r4_points_within.append(p.within(r4))
    r5_points_within.append(p.within(r5))
    r6_points_within.append(p.within(r6))
    r7_points_within.append(p.within(r7))
    #     r8_points_within.append(p.within(r8))
    r9_points_within.append(p.within(r9))
r2_points_within = list(np.asarray(r2_1_points_within) | np.asarray(r2_2_points_within))
# Combine points within and region corners into lists
points_within_regions = [
    r1_points_within,
    r2_points_within,
    r3_points_within,
    r4_points_within,
    r5_points_within,
    r6_points_within,
    r7_points_within,
    r9_points_within,
]
ukca_regions_corners = [
    [r1_llon_ukca, r1_ulon_ukca, r1_llat, r1_ulat],
    [r2_llon_ukca, r2_ulon_ukca, r2_llat, r2_ulat],
    [r3_llon_ukca, r3_ulon_ukca, r3_llat, r3_ulat],
    [r4_llon_ukca, r4_ulon_ukca, r4_llat, r4_ulat],
    [r5_llon_ukca, r5_ulon_ukca, r5_llat, r5_ulat],
    [r6_llon_ukca, r6_ulon_ukca, r6_llat, r6_ulat],
    [r7_llon_ukca, r7_ulon_ukca, r7_llat, r7_ulat],
    [r9_llon_ukca, r9_ulon_ukca, r9_llat, r9_ulat],
]
region_names = [
    "N Pacific",
    "Central Pacific",
    "S Pacific",
    "SE Atlantic",
    "Central Atlantic",
    "N Atlantic",
    "Greenland",
    "W Canada",
]

#### Process ATom data

In [10]:
# Convert ATom C2H6, C2H4, C2H2, C3H8, C3H6 to pptvC to match UKCA lumped species, namely C2H6=C2H6+C2H4+C2H2, C3H8=C3H8+C3H6
atom_dswas_c2h6 = (
    atom_dswas["Ethane_WAS"] * 2 + atom_dswas["Ethene_WAS"] * 2 + atom_dswas["Ethyne_WAS"] * 2
)
atom_dswas_c3h8 = atom_dswas["Propane_WAS"]  # *3 + atom_dswas['Propene_WAS']*3
# Convert xarray dataset with observational data to pandas dataframe
atom_ch4 = atom_dsch4.CH4_NOAA.to_dataframe(name="ch4")
atom_c2h6 = atom_dswas_c2h6.to_dataframe(name="c2h6")
atom_c3h8 = atom_dswas_c3h8.to_dataframe(name="c3h8")
atom_o3 = atom_dso3["O3_CL"].to_dataframe(name="o3")
atom_oh = atom_dsoh["OH_pptv"].to_dataframe(name="oh")
atom_no = atom_dsno["NO_CL"].to_dataframe(name="no")
atom_dsnox = atom_dsno["NO_CL"] + atom_dsno2["NO2_CL"]
atom_nox = atom_dsnox.to_dataframe(name="nox")
atom_noy = atom_dsnoy["NOy_CL"].to_dataframe(name="noy")
atom_meono2 = atom_dswas["MeONO2_WAS"].to_dataframe(name="meono2")
atom_etono2 = atom_dswas["EtONO2_WAS"].to_dataframe(name="etono2")
atom_nprono2 = atom_dswas["n-PrONO2_WAS"].to_dataframe(name="nprono2")
atom_iprono2 = atom_dswas["i-PrONO2_WAS"].to_dataframe(name="iprono2")

In [11]:
# Select longitudes and latitudes where ATom data is available
lons_ch4_notnan = np.where(atom_ch4.ch4.isna() == False, sample_lons, atom_ch4.ch4)
lats_ch4_notnan = np.where(atom_ch4.ch4.isna() == False, sample_lats, atom_ch4.ch4)
lons_c2h6_notnan = np.where(atom_c2h6.c2h6.isna() == False, sample_lons, atom_c2h6.c2h6)
lats_c2h6_notnan = np.where(atom_c2h6.c2h6.isna() == False, sample_lats, atom_c2h6.c2h6)
lons_c3h8_notnan = np.where(atom_c3h8.c3h8.isna() == False, sample_lons, atom_c3h8.c3h8)
lats_c3h8_notnan = np.where(atom_c3h8.c3h8.isna() == False, sample_lats, atom_c3h8.c3h8)
lons_o3_notnan = np.where(atom_o3.o3.isna() == False, sample_lons, atom_o3.o3)
lats_o3_notnan = np.where(atom_o3.o3.isna() == False, sample_lats, atom_o3.o3)
lons_oh_notnan = np.where(atom_oh.oh.isna() == False, sample_lons, atom_oh.oh)
lats_oh_notnan = np.where(atom_oh.oh.isna() == False, sample_lats, atom_oh.oh)
lons_no_notnan = np.where(atom_no.no.isna() == False, sample_lons, atom_no.no)
lats_no_notnan = np.where(atom_no.no.isna() == False, sample_lats, atom_no.no)
lons_nox_noxtnan = np.where(atom_nox.nox.isna() == False, sample_lons, atom_nox.nox)
lats_nox_noxtnan = np.where(atom_nox.nox.isna() == False, sample_lats, atom_nox.nox)
lons_noy_noytnan = np.where(atom_noy.noy.isna() == False, sample_lons, atom_noy.noy)
lats_noy_noytnan = np.where(atom_noy.noy.isna() == False, sample_lats, atom_noy.noy)
lons_meono2_notnan = np.where(atom_meono2.meono2.isna() == False, sample_lons, atom_meono2.meono2)
lats_meono2_notnan = np.where(atom_meono2.meono2.isna() == False, sample_lats, atom_meono2.meono2)
lons_etono2_notnan = np.where(atom_etono2.etono2.isna() == False, sample_lons, atom_etono2.etono2)
lats_etono2_notnan = np.where(atom_etono2.etono2.isna() == False, sample_lats, atom_etono2.etono2)
lons_nprono2_notnan = np.where(
    atom_nprono2.nprono2.isna() == False, sample_lons, atom_nprono2.nprono2
)
lats_nprono2_notnan = np.where(
    atom_nprono2.nprono2.isna() == False, sample_lats, atom_nprono2.nprono2
)
lons_iprono2_notnan = np.where(
    atom_iprono2.iprono2.isna() == False, sample_lons, atom_iprono2.iprono2
)
lats_iprono2_notnan = np.where(
    atom_iprono2.iprono2.isna() == False, sample_lats, atom_iprono2.iprono2
)
lons_notnan = [
    lons_ch4_notnan,
    lons_c2h6_notnan,
    lons_c3h8_notnan,
    lons_o3_notnan,
    lons_oh_notnan,
    lons_no_notnan,
    lons_nox_noxtnan,
    lons_meono2_notnan,
    lons_etono2_notnan,
    lons_nprono2_notnan,
    lons_iprono2_notnan,
]
lats_notnan = [
    lats_ch4_notnan,
    lats_c2h6_notnan,
    lats_c3h8_notnan,
    lats_o3_notnan,
    lats_oh_notnan,
    lats_no_notnan,
    lats_nox_noxtnan,
    lats_meono2_notnan,
    lats_etono2_notnan,
    lats_nprono2_notnan,
    lats_iprono2_notnan,
]

In [12]:
# Bin observational data
alt_bins = np.arange(0, 14000, 500)
alt_bin_inds = np.arange(0, len(alt_bins) - 1, 1)
alt_bin_mids = np.arange(250, 13500, 500)
alt_r1_bin_inds = np.digitize(atom_dsmms.G_ALT.data[r1_points_within], alt_bins)
alt_r2_bin_inds = np.digitize(atom_dsmms.G_ALT.data[r2_points_within], alt_bins)
alt_r3_bin_inds = np.digitize(atom_dsmms.G_ALT.data[r3_points_within], alt_bins)
alt_r4_bin_inds = np.digitize(atom_dsmms.G_ALT.data[r4_points_within], alt_bins)
alt_r5_bin_inds = np.digitize(atom_dsmms.G_ALT.data[r5_points_within], alt_bins)
alt_r6_bin_inds = np.digitize(atom_dsmms.G_ALT.data[r6_points_within], alt_bins)
alt_r7_bin_inds = np.digitize(atom_dsmms.G_ALT.data[r7_points_within], alt_bins)
# alt_r8_bin_inds = np.digitize(atom_dsmms.G_ALT.data[r8_points_within], alt_bins)
alt_r9_bin_inds = np.digitize(atom_dsmms.G_ALT.data[r9_points_within], alt_bins)
alt_regions_bin_inds = [
    alt_r1_bin_inds,
    alt_r2_bin_inds,
    alt_r3_bin_inds,
    alt_r4_bin_inds,
    alt_r5_bin_inds,
    alt_r6_bin_inds,
    alt_r7_bin_inds,
    alt_r9_bin_inds,
]
# alt_regions_bin_inds = [alt_r1_bin_inds, alt_r2_bin_inds, alt_r3_bin_inds,
#                         alt_r4_bin_inds, alt_r5_bin_inds, alt_r6_bin_inds,
#                         alt_r7_bin_inds, alt_r8_bin_inds, alt_r9_bin_inds]
# Calculate mean and standard deviation
atom2process = [
    atom_ch4,
    atom_c2h6,
    atom_c3h8,
    atom_o3,
    atom_oh,
    atom_no,
    atom_nox,
    atom_noy,
    atom_meono2,
    atom_etono2,
    atom_nprono2,
    atom_iprono2,
]
atom_regional_vps = []
for points_within_region, alt_region_bin_inds in zip(points_within_regions, alt_regions_bin_inds):
    specie_dict = {}
    for specie in atom2process:
        stat_dict = {}
        stat_dict["mean"] = (
            specie[points_within_region]
            .groupby(alt_region_bin_inds.compute())
            .mean()
            .reindex(index=alt_bin_inds, fill_value=np.nan)
        )
        stat_dict["std"] = (
            specie[points_within_region]
            .groupby(alt_region_bin_inds.compute())
            .std()
            .reindex(index=alt_bin_inds, fill_value=np.nan)
        )
        stat_dict["count"] = (
            specie[points_within_region]
            .groupby(alt_region_bin_inds.compute())
            .count()
            .reindex(index=alt_bin_inds, fill_value=np.nan)
        )
        specie_dict[specie.columns[0]] = stat_dict
    atom_regional_vps.append(specie_dict)

#### Store ATom vertical profiles

In [13]:
path_to_prcd = Path.home() / "uea" / "phd" / "models" / "ukca" / "processed" / "vertical_profiles"

In [14]:
# Memo: atom_regional_vps[r]['sp_key']['mean','std_dev']
for ir, vp in enumerate(atom_regional_vps):
    for vp_key, vp_value in vp.items():
        cb_mean = iris.cube.Cube(
            vp_value["mean"].values.squeeze(),
            long_name=vp_key,
            dim_coords_and_dims=[
                (iris.coords.DimCoord(alt_bin_mids, standard_name="altitude", units="m"), 0)
            ],
        )
        cb_mean.attributes["region"] = region_names[ir]
        cb_mean.attributes["method"] = "mean"

        cb_std = iris.cube.Cube(
            vp_value["std"].values.squeeze(),
            long_name=vp_key,
            dim_coords_and_dims=[
                (iris.coords.DimCoord(alt_bin_mids, standard_name="altitude", units="m"), 0)
            ],
        )
        cb_std.attributes["region"] = region_names[ir]
        cb_std.attributes["method"] = "std_dev"
        cb_count = iris.cube.Cube(
            vp_value["count"].values.squeeze(),
            long_name=vp_key,
            dim_coords_and_dims=[
                (iris.coords.DimCoord(alt_bin_mids, standard_name="altitude", units="m"), 0)
            ],
        )
        cb_count.attributes["region"] = region_names[ir]
        cb_count.attributes["method"] = "count"

        iris.save(
            cb_mean, str(path_to_prcd / f"vp_{atom_yrmn}_atom_r{ir+1}_{vp_key}_mean.nc")
        )  # store mean and std in separate files
        iris.save(
            cb_std, str(path_to_prcd / f"vp_{atom_yrmn}_atom_r{ir+1}_{vp_key}_std_dev.nc")
        )  # store mean and std in separate files
        iris.save(
            cb_count, str(path_to_prcd / f"vp_{atom_yrmn}_atom_r{ir+1}_{vp_key}_count.nc")
        )  # store mean and std in separate files

#### Process UKCA data

In [15]:
def mmr_to_vmr(cube_mmr, molar_mass):
    M_air = 28.97  # molar mass of dry air [g mol-1]
    cube_vmr = cube_mmr * (M_air / molar_mass)
    return cube_vmr

In [16]:
# Add month and year coordinates
iris.coord_categorisation.add_month(cb_nyrs_ch4, "time", name="month")
iris.coord_categorisation.add_month(cb_nyrs_c2h6, "time", name="month")
iris.coord_categorisation.add_month(cb_nyrs_c3h8, "time", name="month")
iris.coord_categorisation.add_month(cb_nyrs_o3, "time", name="month")
iris.coord_categorisation.add_month(cb_nyrs_oh, "time", name="month")

iris.coord_categorisation.add_month(cb_nyrs_no, "time", name="month")
iris.coord_categorisation.add_month(cb_nyrs_no2, "time", name="month")
iris.coord_categorisation.add_month(cb_nyrs_hono, "time", name="month")
iris.coord_categorisation.add_month(cb_nyrs_ho2no2, "time", name="month")
iris.coord_categorisation.add_month(cb_nyrs_hno3, "time", name="month")
iris.coord_categorisation.add_month(cb_nyrs_n2o5, "time", name="month")
iris.coord_categorisation.add_month(cb_nyrs_pan, "time", name="month")
iris.coord_categorisation.add_month(cb_nyrs_ppan, "time", name="month")

iris.coord_categorisation.add_month(cb_nyrs_meono2, "time", name="month")
iris.coord_categorisation.add_month(cb_nyrs_etono2, "time", name="month")
iris.coord_categorisation.add_month(cb_nyrs_nprono2, "time", name="month")
iris.coord_categorisation.add_month(cb_nyrs_iprono2, "time", name="month")

# Calculate NOx and NOy
cb_nyrs_nox = mmr_to_vmr(cb_nyrs_no, GASES["no"]["molar_mass"]) + mmr_to_vmr(
    cb_nyrs_no2, GASES["no2"]["molar_mass"]
)
cb_nyrs_noy = (
    mmr_to_vmr(cb_nyrs_no, GASES["no"]["molar_mass"])
    + mmr_to_vmr(cb_nyrs_no2, GASES["no2"]["molar_mass"])
    + mmr_to_vmr(cb_nyrs_hono, GASES["hono"]["molar_mass"])
    + mmr_to_vmr(cb_nyrs_ho2no2, GASES["ho2no2"]["molar_mass"])
    + mmr_to_vmr(cb_nyrs_hno3, GASES["hno3"]["molar_mass"])
    + mmr_to_vmr(cb_nyrs_n2o5, GASES["n2o5"]["molar_mass"])
    + mmr_to_vmr(cb_nyrs_pan, GASES["pan"]["molar_mass"])
    + mmr_to_vmr(cb_nyrs_ppan, GASES["ppan"]["molar_mass"])
    + mmr_to_vmr(cb_nyrs_meono2, GASES["meono2"]["molar_mass"])
    + mmr_to_vmr(cb_nyrs_etono2, GASES["etono2"]["molar_mass"])
    + mmr_to_vmr(cb_nyrs_nprono2, GASES["nprono2"]["molar_mass"])
    + mmr_to_vmr(cb_nyrs_iprono2, GASES["iprono2"]["molar_mass"])
)

# Calculate monthly mean from 8 years of data
cb_ch4 = (
    cb_nyrs_ch4[t0:t1]
    .extract(iris.Constraint(month=ukca_mn))
    .aggregated_by(["month"], iris.analysis.MEAN)
)
cb_c2h6 = (
    cb_nyrs_c2h6[t0:t1]
    .extract(iris.Constraint(month=ukca_mn))
    .aggregated_by(["month"], iris.analysis.MEAN)
)
cb_c3h8 = (
    cb_nyrs_c3h8[t0:t1]
    .extract(iris.Constraint(month=ukca_mn))
    .aggregated_by(["month"], iris.analysis.MEAN)
)
cb_o3 = (
    cb_nyrs_o3[t0:t1]
    .extract(iris.Constraint(month=ukca_mn))
    .aggregated_by(["month"], iris.analysis.MEAN)
)
cb_oh = (
    cb_nyrs_oh[t0:t1]
    .extract(iris.Constraint(month=ukca_mn))
    .aggregated_by(["month"], iris.analysis.MEAN)
)
cb_no = (
    cb_nyrs_no[t0:t1]
    .extract(iris.Constraint(month=ukca_mn))
    .aggregated_by(["month"], iris.analysis.MEAN)
)
cb_nox = (
    cb_nyrs_nox[t0:t1]
    .extract(iris.Constraint(month=ukca_mn))
    .aggregated_by(["month"], iris.analysis.MEAN)
)
cb_nox.rename("nox")

cb_noy = (
    cb_nyrs_noy[t0:t1]
    .extract(iris.Constraint(month=ukca_mn))
    .aggregated_by(["month"], iris.analysis.MEAN)
)
cb_noy.rename("noy")

cb_meono2 = (
    cb_nyrs_meono2[t0:t1]
    .extract(iris.Constraint(month=ukca_mn))
    .aggregated_by(["month"], iris.analysis.MEAN)
)
cb_etono2 = (
    cb_nyrs_etono2[t0:t1]
    .extract(iris.Constraint(month=ukca_mn))
    .aggregated_by(["month"], iris.analysis.MEAN)
)
cb_nprono2 = (
    cb_nyrs_nprono2[t0:t1]
    .extract(iris.Constraint(month=ukca_mn))
    .aggregated_by(["month"], iris.analysis.MEAN)
)
cb_iprono2 = (
    cb_nyrs_iprono2[t0:t1]
    .extract(iris.Constraint(month=ukca_mn))
    .aggregated_by(["month"], iris.analysis.MEAN)
)

In [17]:
# Add coordinate bounds (for calculating area weights)
ukca2process = iris.cube.CubeList(
    [
        cb_ch4,
        cb_c2h6,
        cb_c3h8,
        cb_o3,
        cb_oh,
        cb_no,
        cb_nox,
        cb_noy,
        cb_meono2,
        cb_etono2,
        cb_nprono2,
        cb_iprono2,
    ]
)
for cube in ukca2process:
    for coord in ["longitude", "latitude"]:
        if not cube.coord(coord).has_bounds():
            cube.coord(coord).guess_bounds()

In [18]:
# Calculate area weights for each region
any_cube = cb_meono2
regions = []
for corners in ukca_regions_corners:
    llon, ulon, llat, ulat = corners
    lonlat_constraint = iris.Constraint(
        longitude=lambda cell: llon <= cell <= ulon, latitude=lambda cell: llat <= cell <= ulat
    )
    any_cube_extr = any_cube.extract(lonlat_constraint)
    weights_cube = any_cube_extr.copy(data=iris.analysis.cartography.area_weights(any_cube_extr))
    weights_cube.rename("area_weights")
    regions.append(
        {
            "corners": {k: v for k, v in zip(["llon", "ulon", "llat", "ulat"], corners)},
            "area_weights": weights_cube,
        }
    )

In [19]:
# Calculate regional mean vertical profiles and standart deviations
stats = ("mean", "std_dev")
ukca_regional_vps = []
for region in regions:
    cube_dict = {}
    corners = region["corners"]
    lonlat_constraint = iris.Constraint(
        longitude=lambda cell: corners["llon"] <= cell <= corners["ulon"],
        latitude=lambda cell: corners["llat"] <= cell <= corners["ulat"],
    )
    for cube in ukca2process.extract(lonlat_constraint):
        stat_dict = {}
        for stat in stats:
            cubelist = iris.cube.CubeList()
            for lbound, ubound in zip(alt_bins[:-1], alt_bins[1:]):
                alt_constraint = iris.Constraint(altitude=lambda cell: lbound < cell <= ubound)
                cube_extr = cube.extract(alt_constraint)
                if stat == "mean":
                    kwargs = {"weights": region["area_weights"].extract(alt_constraint).data}
                else:
                    kwargs = {}
                cubelist.append(
                    cube_extr.collapsed(
                        ["longitude", "latitude", "altitude"],
                        getattr(iris.analysis, stat.upper()),
                        **kwargs
                    )
                )
            stat_dict[stat] = cubelist.merge_cube()
        cube_dict[cube.name()] = stat_dict
    ukca_regional_vps.append(cube_dict)

#### Store UKCA vertical profiles

In [20]:
# Memo: ukca_regional_vps[r]['sp_key']{'mean','std_dev'}
for ir, vp in enumerate(ukca_regional_vps):
    for vp_key, vp_value in vp.items():
        for var in ["mean", "std_dev"]:
            iris.save(
                vp_value[var],
                str(path_to_prcd / f"vp_{atom_yrmn}_{EXPERIMENTS[exp]}_r{ir+1}_{vp_key}_{var}.nc"),
            )  # store mean and std in separate files